In [ ]:
import os
import pandas as pd
%matplotlib inline
from newsapi import NewsApiClient
import nltk
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Read your api key environment variable
api_key = os.getenv('news_api')

In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key = api_key)

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_news = newsapi.get_everything(q='bitcoin', language='en', page_size=20, sort_by='relevancy')

In [ ]:
# Fetch the Ethereum news articles
ethereum_news = newsapi.get_everything(q='ethereum', language = 'en', page_size=20, sort_by='relevancy')

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []
for article in bitcoin_news["articles"]:
    big_string = article['title'] + ' ' + article['content']
    sentiment = analyzer.polarity_scores(big_string)
    bitcoin_sentiments.append({
        "compound": sentiment["compound"],
        "positive": sentiment["pos"],
        "negative": sentiment["neg"],
        "neutral": sentiment["neu"],
        "date": article["publishedAt"][:10],
        "text": article["content"]
        })
        
bitcoin_df = pd.DataFrame(bitcoin_sentiments)  
bitcoin_df

In [ ]:
# Create the ethereum sentiment scores DataFrame
ethereum_sentiments = []
for article in ethereum_news["articles"]:
    big_string = article['title'] + ' ' + article['content']
    sentiment = analyzer.polarity_scores(big_string)
    bitcoin_sentiments.append({
        "compound": sentiment["compound"],
        "positive": sentiment["pos"],
        "negative": sentiment["neg"],
        "neutral": sentiment["neu"],
        "date": article["publishedAt"][:10],
        "text": article["content"]
        })

ethereum_df = pd.DataFrame(ethereum_sentiments)  
ethereum_df

In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

In [ ]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum

Q: Which coin had the highest compound score?

A: Ethereum

Q. Which coin had the highest positive score?

A: Ethereum

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Expand the default stopwords list if necessary

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Create a list of the words
    # Convert the words to lowercase
    # Remove the punctuation
    # Remove the stop words
    # Lemmatize Words into root words
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    output = [word.lower() for word in words if word.lower() not in sw]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in output]
    return tokens


In [ ]:
# Create a new tokens column for bitcoin
tokens = [tokenizer(text) for text in bitcoin_df['text']]
bitcoin_df['tokens'] = tokens
bitcoin_df

In [ ]:
# Create a new tokens column for ethereum
tokens2 = [tokenizer(text) for text in ethereum_df['text']]
ethereum_df['tokens'] = tokens2
ethereum_df

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
def big_list_maker(token_lists):
    # Takes a list of tokens from each row from a df column 
    # and returns one large list
    big_list = []
    for list in token_lists:
        for token in list:
            big_list.append(token)
    return big_list

bit_coin_bigrams_count = Counter(ngrams(big_list_maker(bitcoin_df['tokens']), n = 2))
bit_coin_bigrams_count.most_common(10)

In [ ]:
# Generate the Ethereum N-grams where N=2
ethereum_bigrams_count = Counter(ngrams(big_list_maker(ethereum_df['tokens']), n = 2))
ethereum_bigrams_count.most_common(10)

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Get the top 10 words for Bitcoin
bitcoin_word_counts_most_common = token_count(big_list_maker(bitcoin_df['tokens']))
bitcoin_word_counts_most_common

In [ ]:
# Get the top 10 words for Ethereum
ethereum_word_counts_most_common = token_count(big_list_maker(ethereum_df['tokens']))
ethereum_word_counts_most_common

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
token_list = big_list_maker(bitcoin_df['tokens'])
input_words = ' '.join(token_list)
wc = WordCloud(collocations = False).generate(input_words)
plt.axis('off')
plt.title('Bitcoin Word Cloud', fontdict = {'fontsize' : 55})
plt.imshow(wc)

In [ ]:
# Generate the Ethereum word cloud
token_list = big_list_maker(ethereum_df['tokens'])
input_words = ' '.join(token_list)
wc = WordCloud(collocations = False).generate(input_words)
plt.axis('off')
plt.title('Ethereum Word Cloud', fontdict = {'fontsize' : 55})
plt.imshow(wc)

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
bitcoin_big_string = ' '.join(bitcoin_df['text'])

In [ ]:
# Run the NER processor on all of the text
bitcoin_doc = nlp(bitcoin_big_string)

# Add a title to the document
bitcoin_doc.user_data["title"] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(bitcoin_doc, style ='ent')

In [ ]:
# List all Entities
def list_all_entities(doc):
    for ent in doc.ents:
        print(ent.text, ent.label_)
        
list_all_entities(bitcoin_doc)

---

## Ethereum NER

In [ ]:
# Concatenate all of the bitcoin text together
ethereum_big_string = ' '.join(ethereum_df['text'])
ethereum_big_string

In [ ]:
# Run the NER processor on all of the text
ethereum_doc = nlp(ethereum_big_string)

# Add a title to the document
ethereum_doc.user_data["title"] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(ethereum_doc, style ='ent')

In [ ]:
# List all Entities
list_all_entities(ethereum_doc)